# Real Hardware - Noise Model

We make use of a real hardware noise model to construct the following distributions using our Galton Board architecture:

- Simple Binomial/Gaussian
- Exponential
- Hadamard Walk (i.e. Binomial with p=0.5)

Note that the recent updates to Qiskit brought about changes in the qiskit.providers.fake_provider module, such that the individual noise model classes like FakeBrisbane are now only accessible using Qiskit Runtime and an account with an API key. IBM has, however made a GenericBackendV2 class available, which, when given a number of qubits, returns a generic BackendV2 implementation with backend properties randomly sampled from real historical IBM backends. This achieves our goal of using a real-hardware derived noise model, even if we can't directly trace which specific backend it is based on.

The construction of this fake backend will slow runtime of the noise-enabled examples.

We shall follow this procedure:

- use the noise model to produce a simple version of each distribution
- compute distance metrics for each example
- for each example, perform a repeated computation to account for stochastic noise.

In [1]:
import os
os.chdir("..")
print(os.getcwd())

c:\Users\hayde\Desktop\Womanium 2025\quantum_nnl_project


In [2]:
from src.galton_box import *
from src.exponential_box import *
from utils.plot_utils import *
from utils.distance_metrics_utils import *

If errors are incurred at any stage, it is worth restarting the Jupyter kernel and then running from the beginning.

## Step 1: Showing differences in results for a perfect vs noise model simulation, standard Gaussian

In [ ]:
# 1-layer

counts = galton_one_layer() # standard simulation
plot_bins(counts, n=1, shots=1000)

counts = galton_one_layer(noise=True) # with noise model
plot_bins(counts, n=1, shots=1000)

# 2-layer

counts = galton_two_layer() # standard simulation
plot_bins(counts, n=2, shots=1000)

counts = galton_two_layer(noise=True) # with noise model
plot_bins(counts, n=2, shots=1000)

# n-layer (test with n=6)

counts = galton_n_layer(n=6) # standard simulation
plot_bins(counts, n=6, shots=1000, overlay="gaussian")

counts = galton_n_layer(n=6, noise=True) # with noise model
plot_bins(counts, n=6, shots=1000, overlay="gaussian")

## Step 2: Showing differences in results for a perfect vs noise model simulation, exponential

In [ ]:
print("STANDARD SIMULATION:")
res = optimise_layerwise(n=6, target="exponential", scale=0.5, shots=1000) # we aim for beta = 0.5 (or lambda = 2)
counts = biased_galton_n_layer(n=6, shots=1000, thetas=res[0])
plot_bins(counts, n=6, shots=1000, overlay="exponential", scale=0.5)

print("SIMULATED OPTIMISER WITH NOISE MODEL FINAL RESULT:")
res = optimise_layerwise(n=6, target="exponential", scale=0.5, shots=1000) # we aim for beta = 0.5 (or lambda = 2)
counts = biased_galton_n_layer(n=6, shots=1000, thetas=res[0], noise=True)
plot_bins(counts, n=6, shots=1000, overlay="exponential", scale=0.5)

## Step 3: Computing Distance Metrics for a Gaussian, Exponential and Laplace Example

In [ ]:
# Gaussian Example: Compare distance metrics between standard and noise model simulation
#(test with n=6)
counts = galton_n_layer(n=6) # standard simulation
plot_bins(counts, n=6, shots=1000, overlay="gaussian")
print(counts)
print(distribution_distance(counts, target_distribution="binomial", metric="w1", p=0.5))

counts = galton_n_layer(n=6, noise=True) # with noise model
plot_bins(counts, n=6, shots=1000, overlay="gaussian")
print(counts)
print(distribution_distance(counts, target_distribution="binomial", metric="w1", p=0.5))

In [ ]:
# Laplace and Exponential Examples

# Exponential:
print("STANDARD SIMULATION:")
res = optimise_layerwise(n=6, target="exponential", scale=0.5, shots=1000) # we aim for beta = 0.5 (or lambda = 2)
counts = biased_galton_n_layer(n=6, shots=1000, thetas=res[0])
plot_bins(counts, n=6, shots=1000, overlay="exponential", scale=0.5)
print(counts)
print(distribution_distance(counts, target_distribution="exponential", lamda=1/0.5, metric="js"))

print("SIMULATED OPTIMISER WITH NOISE MODEL FINAL RESULT:")
res = optimise_layerwise(n=6, target="exponential", scale=0.5, shots=1000) # we aim for beta = 0.5 (or lambda = 2)
counts = biased_galton_n_layer(n=6, shots=1000, thetas=res[0], noise=True)
plot_bins(counts, n=6, shots=1000, overlay="exponential", scale=0.5)
print(counts)
print(distribution_distance(counts, target_distribution="exponential", lamda=1/0.5, metric="js"))

In [ ]:
# Laplace:
print("STANDARD SIMULATION:")
res = optimise_layerwise(n=6, target="laplace", decay=1, shots=1000) 
counts = biased_galton_n_layer(n=6, shots=1000, thetas=res[0])
plot_bins(counts, n=6, shots=1000, overlay="laplace", scale=1)
print(counts)
print(distribution_distance(counts, target_distribution="laplace", mu=6//2, b=1, metric="js"))

print("SIMULATED OPTIMISER WITH NOISE MODEL FINAL RESULT:")
res = optimise_layerwise(n=6, target="laplace", decay=1, shots=1000) 
counts = biased_galton_n_layer(n=6, shots=1000, thetas=res[0], noise=True)
plot_bins(counts, n=6, shots=1000, overlay="laplace", scale=1)
print(counts)
print(distribution_distance(counts, target_distribution="laplace", mu=6//2, b=1, metric="js"))

## Step 4: Accounting for Stochastic Noise Using Multiple Averaged Runs - t-tests

In [5]:
# Gaussian - Small Example
# Perfect Sim:
print("STANDARD SIMULATION")
distance_list = []
for i in range(50):
    counts = galton_n_layer(n=4, shots=1000)
    print(counts)
    distance_list.append(distribution_distance(counts, target_distribution="gaussian", metric="w1", p=0.5))
print(distance_list)
print(significance_test(distance_list, test_type="t", significance_level=0.05, threshold=0.1))
print(f"\n\n\n")

# Averaged noise model:
print("NOISE MODEL")
distance_list = []
for i in range(30):
    counts = galton_n_layer(n=4, shots=1000, noise=True) # with noise model
    print(counts)
    distance_list.append(distribution_distance(counts, target_distribution="gaussian", metric="w1", p=0.5))
print(distance_list)
print(significance_test(distance_list, test_type="t", significance_level=0.05, threshold=0.1))

STANDARD SIMULATION
{0: 64, 1: 248, 2: 379, 3: 258, 4: 51}
{0: 72, 1: 257, 2: 374, 3: 233, 4: 64}
{0: 60, 1: 243, 2: 403, 3: 231, 4: 63}
{0: 59, 1: 260, 2: 369, 3: 253, 4: 59}
{0: 67, 1: 253, 2: 372, 3: 237, 4: 71}
{0: 54, 1: 260, 2: 356, 3: 267, 4: 63}
{0: 59, 1: 253, 2: 394, 3: 249, 4: 45}
{0: 56, 1: 240, 2: 402, 3: 237, 4: 65}
{0: 67, 1: 231, 2: 380, 3: 250, 4: 72}
{0: 66, 1: 260, 2: 388, 3: 213, 4: 73}
{0: 50, 1: 254, 2: 380, 3: 257, 4: 59}
{0: 58, 1: 258, 2: 372, 3: 242, 4: 70}
{0: 65, 1: 249, 2: 350, 3: 274, 4: 62}
{0: 53, 1: 248, 2: 365, 3: 261, 4: 73}
{0: 65, 1: 259, 2: 372, 3: 242, 4: 62}
{0: 58, 1: 232, 2: 378, 3: 286, 4: 46}
{0: 70, 1: 261, 2: 344, 3: 245, 4: 80}
{0: 62, 1: 269, 2: 352, 3: 248, 4: 69}
{0: 65, 1: 244, 2: 407, 3: 224, 4: 60}
{0: 70, 1: 233, 2: 374, 3: 250, 4: 73}
{0: 69, 1: 234, 2: 374, 3: 267, 4: 56}
{0: 62, 1: 243, 2: 402, 3: 241, 4: 52}
{0: 61, 1: 269, 2: 365, 3: 244, 4: 61}
{0: 70, 1: 252, 2: 359, 3: 254, 4: 65}
{0: 73, 1: 259, 2: 363, 3: 266, 4: 39}
{0: 6

## Step 5: Accounting for Stochastic Noise Using Multiple Averaged Runs - Chi-Squared Tests

In [6]:
# Gaussian - Small Example
# Perfect Sim:
print("STANDARD SIMULATION")
distance_list = []
for i in range(50):
    counts = galton_n_layer(n=4, shots=1000)
    print(counts)
    distance_list.append(distribution_distance(counts, target_distribution="gaussian", metric="chi", p=0.5))
print(distance_list)
print(significance_test(distance_list, test_type="chi", significance_level=0.05, n=4))
print(f"\n\n\n")

# Averaged noise model:
print("NOISE MODEL")
distance_list = []
for i in range(30):
    counts = galton_n_layer(n=4, shots=1000, noise=True) # with noise model
    print(counts)
    distance_list.append(distribution_distance(counts, target_distribution="gaussian", metric="chi", p=0.5))
print(distance_list)
print(significance_test(distance_list, test_type="chi", significance_level=0.05, n=4))

STANDARD SIMULATION
{0: 71, 1: 228, 2: 377, 3: 270, 4: 54}
{0: 59, 1: 252, 2: 367, 3: 267, 4: 55}
{0: 67, 1: 246, 2: 394, 3: 231, 4: 62}
{0: 56, 1: 243, 2: 352, 3: 281, 4: 68}
{0: 61, 1: 255, 2: 397, 3: 229, 4: 58}
{0: 56, 1: 237, 2: 392, 3: 257, 4: 58}
{0: 54, 1: 239, 2: 389, 3: 252, 4: 66}
{0: 59, 1: 259, 2: 374, 3: 252, 4: 56}
{0: 67, 1: 240, 2: 400, 3: 235, 4: 58}
{0: 59, 1: 254, 2: 389, 3: 235, 4: 63}
{0: 57, 1: 253, 2: 371, 3: 267, 4: 52}
{0: 59, 1: 242, 2: 364, 3: 262, 4: 73}
{0: 49, 1: 267, 2: 376, 3: 244, 4: 64}
{0: 82, 1: 246, 2: 361, 3: 246, 4: 65}
{0: 68, 1: 258, 2: 356, 3: 262, 4: 56}
{0: 54, 1: 259, 2: 399, 3: 231, 4: 57}
{0: 56, 1: 259, 2: 380, 3: 246, 4: 59}
{0: 63, 1: 247, 2: 363, 3: 253, 4: 74}
{0: 63, 1: 255, 2: 365, 3: 250, 4: 67}
{0: 54, 1: 237, 2: 391, 3: 258, 4: 60}
{0: 55, 1: 230, 2: 387, 3: 248, 4: 80}
{0: 68, 1: 235, 2: 377, 3: 259, 4: 61}
{0: 69, 1: 238, 2: 369, 3: 260, 4: 64}
{0: 70, 1: 242, 2: 380, 3: 238, 4: 70}
{0: 64, 1: 252, 2: 382, 3: 237, 4: 65}
{0: 6